In [1]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import pickle 
import heapq
# nlp concept
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
# from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop

In [3]:
path = '1661-0.txt'
text = open(path, encoding="utf8").read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [4]:
#Split the dataset into each word in order, without presence of some special characters,

tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

# Feature Engineering

In [5]:
#unique list
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [6]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


# Model Building using RNN

In [7]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [8]:
print(X[0][0])

[False False False ... False False False]


In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

d:\Users\USER\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


# Training

In [10]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 136s 163ms/step - accuracy: 0.0546 - loss: 6.6324 - val_accuracy: 0.0820 - val_loss: 6.9524
Epoch 2/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 129s 160ms/step - accuracy: 0.1081 - loss: 5.8384 - val_accuracy: 0.0864 - val_loss: 6.7530
Epoch 3/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 129s 159ms/step - accuracy: 0.1334 - loss: 5.5126 - val_accuracy: 0.0979 - val_loss: 6.7054
Epoch 4/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 132s 162ms/step - accuracy: 0.1471 - loss: 5.2586 - val_accuracy: 0.0985 - val_loss: 6.6304
Epoch 5/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 132s 162ms/step - accuracy: 0.1683 - loss: 5.0063 - val_accuracy: 0.1056 - val_loss: 6.6016
Epoch 6/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 130s 160ms/step - accuracy: 0.1931 - loss: 4.7467 - val_accuracy: 0.0996 - val_loss: 6.6896
Epoch 7/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 129s 159ms/step - accuracy: 0.2232 - loss: 4.4453 - val_accuracy: 0.0895 - val_loss: 6.7475
Epoch 8/20
811/811 ━━━━━━━━━━━━━━━━━━━━ 133s 164ms/step - accuracy: 0.2598 -

# Saving Model

In [11]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [12]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("How are you ".lower())

how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [13]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [14]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

# Testing 

In [15]:
q =  "Do your work by your own instead of depending on someone"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Do your work by your own instead of depending on someone
Sequence:  do your work by your
do
your
work
by
your
next possible words:  ['way', 'position', 'wife', 'time', 'experience']


In [16]:
q =  "Do your work by your own instead of depending on someone"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  Do your work by your own instead of depending on someone
Sequence:  do your work by your
do
your
work
by
your
next possible words:  ['way', 'position', 'wife', 'time', 'experience']
